In [1]:
import math
from sklearn import neighbors, svm
import os
import os.path
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
import cv2
from matplotlib.pyplot import imshow 

In [2]:
train_dir = "ActorsDataset"

trainX = []
trainY = []
testX = []
testY = []

In [3]:
cell_size = (16, 16)  # h x w in pixels
block_size = (2, 2)  # h x w in cells
nbins = 9  # number of orientation bins

hog = cv2.HOGDescriptor(_winSize=(64 // cell_size[1] * cell_size[1],
                          64 // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)

for class_dir in os.listdir(train_dir):

    nr = 0
    for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
        
        image = face_recognition.load_image_file(img_path)
        face_bounding_boxes = face_recognition.face_locations(image)
        
        if len(face_bounding_boxes) == 1:
            (top, right, bottom, left) = face_bounding_boxes[0]
            face_img = cv2.resize(image[top:bottom, left:right],(64,64))
            img_hog = hog.compute(face_img)
            #imshow(face_img)
            #nx, ny = trainX[1].shape
            #d2_train_dataset = trainX[0].reshape(nx*ny)
            if nr < 20:
                testX.append(img_hog.reshape(324))
                testY.append(class_dir)
            else:
                trainX.append(img_hog.reshape(324))
                trainY.append(class_dir)
            nr = nr + 1

In [4]:
print(len(trainX))
print(len(trainY))
print(len(testX))
print(len(testY))

8654
8654
2000
2000


In [5]:
def trainKNN(model_save_path=None, n_neighbors=None, knn_algo='ball_tree'):
    
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(trainX))))

    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(trainX, trainY)

    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

In [18]:
print("Training KNN classifier...")
knn_clf = trainKNN(n_neighbors=11)
print("Training complete!")

Training KNN classifier...
Training complete!


In [19]:
def predictKNN(faces_encodings, distance_threshold=0.6):

    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][0][0] <= distance_threshold]
    
    if are_matches:
        return knn_clf.predict(faces_encodings)[0]
    else:
        return "unkown"

In [20]:
total = 0
nrCorecte = 0
for i in range(len(testX)):
    name = predictKNN([testX[i]])
    total = total + 1
    if name == testY[i]:
        nrCorecte = nrCorecte + 1
    else:
        print("Corect: " + testY[i] + "; Prezis: " + name)

print("Corecte KNN: " + str(nrCorecte))
print("Totale KNN: " + str(total))
print("Acc KNN: " + str(nrCorecte/total))

Corect: Madchen Amick; Prezis: Shailene Woodley
Corect: Madchen Amick; Prezis: Tom Holland
Corect: Madchen Amick; Prezis: Serinda Swan
Corect: Madchen Amick; Prezis: Gal Gadot
Corect: Madchen Amick; Prezis: Alison Brie
Corect: Madchen Amick; Prezis: Jessica Henwick
Corect: Madchen Amick; Prezis: Alison Brie
Corect: Madchen Amick; Prezis: Travis Fimmel
Corect: Madchen Amick; Prezis: Dan Stevens
Corect: Madchen Amick; Prezis: Brie Larson
Corect: Madchen Amick; Prezis: Cara Delevingne
Corect: Madchen Amick; Prezis: Connie Nielsen
Corect: Madchen Amick; Prezis: Luke Evans
Corect: Madchen Amick; Prezis: Lena Headey
Corect: Madchen Amick; Prezis: Amy Adams
Corect: Madchen Amick; Prezis: Elizabeth Debicki
Corect: Madchen Amick; Prezis: Leonardo Dicaprio
Corect: Madchen Amick; Prezis: Charlize Theron
Corect: Jessica Henwick; Prezis: Emma Watson
Corect: Jessica Henwick; Prezis: Yvonne Strahovski
Corect: Jessica Henwick; Prezis: Ana de Armas
Corect: Jessica Henwick; Prezis: Nina Dobrev
Corect: J

Corect: Travis Fimmel; Prezis: Ryan Gosling
Corect: Ryan Gosling; Prezis: Chris Pratt
Corect: Ryan Gosling; Prezis: Tom Cruise
Corect: Ryan Gosling; Prezis: Alexander Skarsgard
Corect: Ryan Gosling; Prezis: Charlie Hunnam
Corect: Ryan Gosling; Prezis: Amy Adams
Corect: Ryan Gosling; Prezis: Chris Hemsworth
Corect: Ryan Gosling; Prezis: Tom Hardy
Corect: Ryan Gosling; Prezis: Charlie Hunnam
Corect: Ryan Gosling; Prezis: Dan Stevens
Corect: Ryan Gosling; Prezis: Chris Pine
Corect: Ryan Gosling; Prezis: Chris Pine
Corect: Ryan Gosling; Prezis: Bill Paxton
Corect: Ryan Gosling; Prezis: Alexander Skarsgard
Corect: Ryan Gosling; Prezis: Johnny Depp
Corect: Ryan Gosling; Prezis: Charlie Hunnam
Corect: Elizabeth Debicki; Prezis: Brie Larson
Corect: Elizabeth Debicki; Prezis: Pom Klementieff
Corect: Elizabeth Debicki; Prezis: Scarlett Johansson
Corect: Elizabeth Debicki; Prezis: Johnny Depp
Corect: Elizabeth Debicki; Prezis: Katheryn Winnick
Corect: Elizabeth Debicki; Prezis: Allison Williams
C

Corect: Rosamund Pike; Prezis: Ryan Gosling
Corect: Rosamund Pike; Prezis: Lily James
Corect: Jason Momoa; Prezis: Chris Pine
Corect: Jason Momoa; Prezis: Chris Hemsworth
Corect: Jason Momoa; Prezis: Charlie Hunnam
Corect: Jason Momoa; Prezis: Tom Hardy
Corect: Jason Momoa; Prezis: Dan Stevens
Corect: Jason Momoa; Prezis: Charlie Hunnam
Corect: Jason Momoa; Prezis: Madchen Amick
Corect: Zoe Saldana; Prezis: Charlize Theron
Corect: Zoe Saldana; Prezis: Nina Dobrev
Corect: Zoe Saldana; Prezis: Chris Pine
Corect: Zoe Saldana; Prezis: Chris Pratt
Corect: Zoe Saldana; Prezis: Daisy Ridley
Corect: Zoe Saldana; Prezis: Sophia Lillis
Corect: Zoe Saldana; Prezis: Shailene Woodley
Corect: Zoe Saldana; Prezis: Ana de Armas
Corect: Zoe Saldana; Prezis: Ana de Armas
Corect: Zoe Saldana; Prezis: Luke Evans
Corect: Zoe Saldana; Prezis: Scarlett Johansson
Corect: Zoe Saldana; Prezis: Daisy Ridley
Corect: Zoe Saldana; Prezis: Chloe Grace Moretz
Corect: Zoe Saldana; Prezis: Eiza Gonzalez
Corect: Nina Do

Corect: Ana de Armas; Prezis: Rose Leslie
Corect: Ana de Armas; Prezis: Alexandra Daddario
Corect: Ana de Armas; Prezis: Kate Beckinsale
Corect: Ana de Armas; Prezis: Emily Browning
Corect: Ana de Armas; Prezis: Tom Hardy
Corect: Ana de Armas; Prezis: Eiza Gonzalez
Corect: Ana de Armas; Prezis: Annabelle Wallis
Corect: Ana de Armas; Prezis: Margot Robbie
Corect: Ana de Armas; Prezis: Rose Leslie
Corect: Ana de Armas; Prezis: Scarlett Johansson
Corect: Ana de Armas; Prezis: Daisy Ridley
Corect: Ana de Armas; Prezis: Dakota Johnson
Corect: Ana de Armas; Prezis: Katherine Waterston
Corect: Ana de Armas; Prezis: Alexander Skarsgard
Corect: Jodie Whittaker; Prezis: Kate Beckinsale
Corect: Jodie Whittaker; Prezis: Allison Williams
Corect: Jodie Whittaker; Prezis: Emily Browning
Corect: Jodie Whittaker; Prezis: Julia Garner
Corect: Jodie Whittaker; Prezis: Charlie Hunnam
Corect: Jodie Whittaker; Prezis: Dakota Johnson
Corect: Jodie Whittaker; Prezis: Malina Weissman
Corect: Jodie Whittaker; P

Corect: Millie Bobby Brown; Prezis: Bill Skarsgard
Corect: Millie Bobby Brown; Prezis: Hugh Jackman
Corect: Millie Bobby Brown; Prezis: Emily Browning
Corect: Millie Bobby Brown; Prezis: Luke Evans
Corect: Millie Bobby Brown; Prezis: Shailene Woodley
Corect: Millie Bobby Brown; Prezis: Chris Pine
Corect: Millie Bobby Brown; Prezis: Tom Cruise
Corect: Millie Bobby Brown; Prezis: Tom Hardy
Corect: Millie Bobby Brown; Prezis: Chris Pine
Corect: Millie Bobby Brown; Prezis: Madchen Amick
Corect: Millie Bobby Brown; Prezis: Daisy Ridley
Corect: Millie Bobby Brown; Prezis: Yvonne Strahovski
Corect: Millie Bobby Brown; Prezis: Teresa Palmer
Corect: Millie Bobby Brown; Prezis: Yvonne Strahovski
Corect: Millie Bobby Brown; Prezis: Bill Skarsgard
Corect: Millie Bobby Brown; Prezis: Britt Robertson
Corect: Millie Bobby Brown; Prezis: Madchen Amick
Corect: Millie Bobby Brown; Prezis: Alison Brie
Corect: Brie Larson; Prezis: Adrianne Palicki
Corect: Brie Larson; Prezis: Margot Robbie
Corect: Brie La

Corect: James McAvoy; Prezis: Tom Hardy
Corect: James McAvoy; Prezis: Alexander Skarsgard
Corect: James McAvoy; Prezis: Chris Pine
Corect: James McAvoy; Prezis: Sylvester Stallone
Corect: James McAvoy; Prezis: Alicia Vikander
Corect: James McAvoy; Prezis: Amy Adams
Corect: James McAvoy; Prezis: Chris Hemsworth
Corect: James McAvoy; Prezis: Chris Pine
Corect: James McAvoy; Prezis: Chris Hemsworth
Corect: James McAvoy; Prezis: Christian Navarro
Corect: James McAvoy; Prezis: Tom Hardy
Corect: James McAvoy; Prezis: Tom Hardy
Corect: James McAvoy; Prezis: Dan Stevens
Corect: James McAvoy; Prezis: Cara Delevingne
Corect: James McAvoy; Prezis: Johnny Depp
Corect: Hugh Jackman; Prezis: Dan Stevens
Corect: Hugh Jackman; Prezis: James McAvoy
Corect: Hugh Jackman; Prezis: Robin Wright
Corect: Hugh Jackman; Prezis: Travis Fimmel
Corect: Hugh Jackman; Prezis: Tom Cruise
Corect: Hugh Jackman; Prezis: Tom Holland
Corect: Hugh Jackman; Prezis: Eiza Gonzalez
Corect: Hugh Jackman; Prezis: Dave Bautista


Corect: Julia Garner; Prezis: Leonardo Dicaprio
Corect: Bella Thorne; Prezis: Haley Bennett
Corect: Bella Thorne; Prezis: Sophie Turner
Corect: Bella Thorne; Prezis: Eiza Gonzalez
Corect: Bella Thorne; Prezis: Pom Klementieff
Corect: Bella Thorne; Prezis: Alison Brie
Corect: Bella Thorne; Prezis: Amy Adams
Corect: Bella Thorne; Prezis: Chloe Grace Moretz
Corect: Bella Thorne; Prezis: Katheryn Winnick
Corect: Bella Thorne; Prezis: Tom Holland
Corect: Bella Thorne; Prezis: Teresa Palmer
Corect: Bella Thorne; Prezis: Elizabeth Debicki
Corect: Bella Thorne; Prezis: Pom Klementieff
Corect: Bella Thorne; Prezis: Kate Mara
Corect: Bella Thorne; Prezis: Allison Williams
Corect: Bella Thorne; Prezis: Halston Sage
Corect: Bella Thorne; Prezis: Yvonne Strahovski
Corect: Bella Thorne; Prezis: Jennifer Lawrence
Corect: Bella Thorne; Prezis: Brie Larson
Corect: Bella Thorne; Prezis: Sophie Turner
Corect: Eiza Gonzalez; Prezis: Rosamund Pike
Corect: Eiza Gonzalez; Prezis: Travis Fimmel
Corect: Eiza G

Corect: Charlize Theron; Prezis: Emilia Clarke
Corect: Charlize Theron; Prezis: Katheryn Winnick
Corect: Charlize Theron; Prezis: Rosamund Pike
Corect: Charlize Theron; Prezis: Jodie Whittaker
Corect: Charlize Theron; Prezis: Katheryn Winnick
Corect: Charlize Theron; Prezis: Zoe Saldana
Corect: Charlize Theron; Prezis: Zoey Deutch
Corect: Charlize Theron; Prezis: Pom Klementieff
Corect: Charlize Theron; Prezis: Jodie Whittaker
Corect: Charlize Theron; Prezis: Cara Delevingne
Corect: Charlize Theron; Prezis: Gal Gadot
Corect: Connie Nielsen; Prezis: Chris Hemsworth
Corect: Connie Nielsen; Prezis: Auli_i Cravalho
Corect: Connie Nielsen; Prezis: Chris Pine
Corect: Connie Nielsen; Prezis: Julia Garner
Corect: Connie Nielsen; Prezis: Tom Cruise
Corect: Connie Nielsen; Prezis: Rosamund Pike
Corect: Connie Nielsen; Prezis: Britt Robertson
Corect: Connie Nielsen; Prezis: Zoe Saldana
Corect: Connie Nielsen; Prezis: Chris Pine
Corect: Connie Nielsen; Prezis: Adrianne Palicki
Corect: Connie Niels

In [63]:
def trainSVM(model_save_path=None):

    svm_clf = svm.SVC(kernel='linear', C = 15.0)
    svm_clf.fit(trainX, trainY)
    
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(svm_clf, f)

    return svm_clf

In [64]:
print("Training SVM classifier...")
svm_clf = trainSVM()
print("Training complete!")

Training SVM classifier...
Training complete!


In [65]:
def predictSVM(faces_encodings):
    return svm_clf.predict(faces_encodings)[0]

In [66]:
total = 0
nrCorecte = 0
for i in range(len(testX)):
    name = predictSVM([testX[i]])
    total = total + 1
    if name == testY[i]:
        nrCorecte = nrCorecte + 1
    else:
        print("Corect: " + testY[i] + "; Prezis: " + name)

print("Corecte KNN: " + str(nrCorecte))
print("Totale KNN: " + str(total))
print("Acc KNN: " + str(nrCorecte/total))

Corect: Madchen Amick; Prezis: Katherine Langford
Corect: Madchen Amick; Prezis: Jodie Whittaker
Corect: Madchen Amick; Prezis: Charlize Theron
Corect: Madchen Amick; Prezis: Johnny Depp
Corect: Madchen Amick; Prezis: Anya Taylor-Joy
Corect: Madchen Amick; Prezis: Lily James
Corect: Madchen Amick; Prezis: Kaya Scodelario
Corect: Madchen Amick; Prezis: Lena Headey
Corect: Madchen Amick; Prezis: Bryce Dallas Howard
Corect: Madchen Amick; Prezis: Chris Pine
Corect: Madchen Amick; Prezis: Julia Garner
Corect: Madchen Amick; Prezis: Lily James
Corect: Jessica Henwick; Prezis: Chloe Grace Moretz
Corect: Jessica Henwick; Prezis: Rosamund Pike
Corect: Jessica Henwick; Prezis: Eiza Gonzalez
Corect: Jessica Henwick; Prezis: Gal Gadot
Corect: Jessica Henwick; Prezis: Alexander Skarsgard
Corect: Jessica Henwick; Prezis: Emily Browning
Corect: Jessica Henwick; Prezis: Kelly Rohrbach
Corect: Jessica Henwick; Prezis: Alicia Vikander
Corect: Jessica Henwick; Prezis: Bill Skarsgard
Corect: Jessica Henw

Corect: Tom Hardy; Prezis: Alexander Skarsgard
Corect: Tom Hardy; Prezis: Kit Harington
Corect: Tom Hardy; Prezis: Dave Bautista
Corect: Ruby Rose; Prezis: Karen Gillan
Corect: Ruby Rose; Prezis: Bella Thorne
Corect: Ruby Rose; Prezis: Ana de Armas
Corect: Ruby Rose; Prezis: Alison Brie
Corect: Ruby Rose; Prezis: Zendaya
Corect: Ruby Rose; Prezis: Tom Cruise
Corect: Elodie Yung; Prezis: Bella Thorne
Corect: Elodie Yung; Prezis: Haley Bennett
Corect: Elodie Yung; Prezis: Scarlett Johansson
Corect: Elodie Yung; Prezis: Rosamund Pike
Corect: Elodie Yung; Prezis: Emilia Clarke
Corect: Elodie Yung; Prezis: Ruby Rose
Corect: Elodie Yung; Prezis: Sofia Boutella
Corect: Elodie Yung; Prezis: Alexandra Daddario
Corect: Elodie Yung; Prezis: Dafne Keen
Corect: Gal Gadot; Prezis: Mary Elizabeth Winstead
Corect: Gal Gadot; Prezis: Jessica Henwick
Corect: Gal Gadot; Prezis: Ruby Rose
Corect: Gal Gadot; Prezis: Dafne Keen
Corect: Gal Gadot; Prezis: Kate Mara
Corect: Gal Gadot; Prezis: Abigail Breslin


Corect: Lily James; Prezis: Adrianne Palicki
Corect: Lily James; Prezis: Madchen Amick
Corect: Lily James; Prezis: Brie Larson
Corect: Lily James; Prezis: Elodie Yung
Corect: Lily James; Prezis: Katheryn Winnick
Corect: Lily James; Prezis: Finn Jones
Corect: Lily James; Prezis: Abigail Breslin
Corect: Kaya Scodelario; Prezis: Dafne Keen
Corect: Kaya Scodelario; Prezis: Katheryn Winnick
Corect: Kaya Scodelario; Prezis: Pom Klementieff
Corect: Kaya Scodelario; Prezis: James McAvoy
Corect: Kaya Scodelario; Prezis: Rebecca Ferguson
Corect: Kaya Scodelario; Prezis: Eiza Gonzalez
Corect: Kaya Scodelario; Prezis: Madchen Amick
Corect: Kaya Scodelario; Prezis: Eiza Gonzalez
Corect: Kaya Scodelario; Prezis: Rebecca Ferguson
Corect: Kaya Scodelario; Prezis: Emilia Clarke
Corect: Kaya Scodelario; Prezis: Dafne Keen
Corect: Auli_i Cravalho; Prezis: Zoe Saldana
Corect: Auli_i Cravalho; Prezis: Annabelle Wallis
Corect: Auli_i Cravalho; Prezis: Kaya Scodelario
Corect: Auli_i Cravalho; Prezis: Kelly R

Corect: Katheryn Winnick; Prezis: Teresa Palmer
Corect: Katheryn Winnick; Prezis: Julia Garner
Corect: Katheryn Winnick; Prezis: Teresa Palmer
Corect: Katheryn Winnick; Prezis: Emily Browning
Corect: Cole Sprouse; Prezis: Brie Larson
Corect: Cole Sprouse; Prezis: Chloe Grace Moretz
Corect: Cole Sprouse; Prezis: James McAvoy
Corect: Cole Sprouse; Prezis: Dylan Minnette
Corect: Cole Sprouse; Prezis: Adrianne Palicki
Corect: Cole Sprouse; Prezis: Charlie Hunnam
Corect: Cole Sprouse; Prezis: Annabelle Wallis
Corect: Cole Sprouse; Prezis: Dave Bautista
Corect: Cole Sprouse; Prezis: Sean Young
Corect: Cole Sprouse; Prezis: Sylvester Stallone
Corect: Cole Sprouse; Prezis: Tom Cruise
Corect: Cole Sprouse; Prezis: Charlize Theron
Corect: Cole Sprouse; Prezis: Katherine Langford
Corect: Cole Sprouse; Prezis: Rose Leslie
Corect: Cole Sprouse; Prezis: Kit Harington
Corect: James McAvoy; Prezis: Leonardo Dicaprio
Corect: James McAvoy; Prezis: Jason Momoa
Corect: James McAvoy; Prezis: Tom Hardy
Core

Corect: Kit Harington; Prezis: Finn Jones
Corect: Daisy Ridley; Prezis: Abigail Breslin
Corect: Daisy Ridley; Prezis: Eiza Gonzalez
Corect: Daisy Ridley; Prezis: Cole Sprouse
Corect: Daisy Ridley; Prezis: Annabelle Wallis
Corect: Daisy Ridley; Prezis: Madchen Amick
Corect: Daisy Ridley; Prezis: Allison Williams
Corect: Daisy Ridley; Prezis: Kate Mara
Corect: Daisy Ridley; Prezis: Katheryn Winnick
Corect: Daisy Ridley; Prezis: Madchen Amick
Corect: Daisy Ridley; Prezis: Allison Williams
Corect: Daisy Ridley; Prezis: Nina Dobrev
Corect: Daisy Ridley; Prezis: Nina Dobrev
Corect: Daisy Ridley; Prezis: Lily James
Corect: Kelly Rohrbach; Prezis: Emma Stone
Corect: Kelly Rohrbach; Prezis: Rebecca Ferguson
Corect: Kelly Rohrbach; Prezis: Cara Delevingne
Corect: Kelly Rohrbach; Prezis: Emily Browning
Corect: Kelly Rohrbach; Prezis: Zoey Deutch
Corect: Kelly Rohrbach; Prezis: Karen Gillan
Corect: Kelly Rohrbach; Prezis: Teresa Palmer
Corect: Kelly Rohrbach; Prezis: Cara Delevingne
Corect: Kelly 